Notebook for processing metadata tables for use in silver. 

NOTE: partially complete b/c accidently lost the notebook

In [3]:
spark.catalog.listTables('test.silver.md') # see metadata tables we made

[Table(name='binary', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='largecat', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='number', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='smallcat', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False)]

In [4]:
import numpy as np # for NULL cast since pandas makes NULL nan
sdf = spark.table("test.silver.md.smallcat").toPandas()
unique_fields = np.unique(sdf['field'])
print(sdf, unique_fields)

   silver_column field  code  \
0      DRAT_NULL  DRAT   NaN   
1         DRAT_1  DRAT   1.0   
2         DRAT_2  DRAT   2.0   
3         DRAT_3  DRAT   3.0   
4         DRAT_4  DRAT   4.0   
..           ...   ...   ...   
60        WAOB_4  WAOB   4.0   
61        WAOB_5  WAOB   5.0   
62        WAOB_6  WAOB   6.0   
63        WAOB_7  WAOB   7.0   
64        WAOB_8  WAOB   8.0   

                                                texts  
0   N/A (No service-connected disability/never ser...  
1                                           0 percent  
2                                    10 or 20 percent  
3                                    30 or 40 percent  
4                                    50 or 60 percent  
..                                                ...  
60                  Asia (POBP = 158-159,161,200-299)  
61                Europe (POBP = 100-157,160,162-199)  
62                            Africa (POBP = 400-499)  
63          Northern America (POBP = 300-302,304-309)  

In [9]:
query = "SELECT YEAR, PUMA"
for field in unique_fields:
    if field == 'REGION': # FIXME
        continue
    query += f",\n{field}"
for _, row in sdf.iterrows():
    if row['field'] == 'REGION': # FIXME
        continue
    silver_column, field, code, text = row.values
    match_str = '= ' + str(code) if not np.isnan(code) else 'IS NULL'
    query += f",\nCASE WHEN {field} {match_str} THEN 1 ELSE 0 END {silver_column}"
query += "\nFROM test.bronze.census"
spark.sql(query).writeTo("test.silver.data.smallcat").createOrReplace()

In [13]:
%%sql

SELECT * FROM test.silver.data.smallcat

agep,marhyp,oip,pap,pernp,pincp,pwgtp,retp,wagp,wkhp,wkwn,yoep
48,2009,0,0,0,0,59,0,0,None,None,None
19,None,0,0,0,0,36,0,0,None,None,None
36,2012,2500,0,0,2500,23,0,0,None,None,None
27,None,0,0,31300,31300,42,0,31300,48,48,None
92,1950,0,0,0,22000,14,6000,0,None,None,None
19,None,0,0,0,0,53,0,0,None,None,None
19,None,0,0,6000,6000,32,0,6000,50,1,None
81,1961,0,0,0,10100,56,0,0,None,None,None
92,1949,0,0,0,9300,62,6600,0,None,None,None
31,None,0,0,0,0,54,0,0,None,None,None


In [ ]:
%%sql

